# Import Libraries

In [4]:
import pandas as pd
import numpy as np

# Obtain Data

In [5]:
df = pd.read_csv('Data/400m_Hurdles_WR_Progression.csv')
print('df shape: ', df.shape)
df.head()

df shape:  (23, 7)


,Mark,Wind,Competitor,DOB,Country,Venue,Date
0,45.94,NaN,Karsten WARHOLM,28 FEB 1996,NOR,"Olympic Stadium, Tokyo (JPN)",03 AUG 2021
1,46.70,NaN,Karsten WARHOLM,28 FEB 1996,NOR,"Bislett Stadion, Oslo (NOR)",01 JUL 2021
2,46.78,NaN,Kevin YOUNG,16 SEP 1966,USA,"Estadio Olímpico, Barcelona (ESP)",06 AUG 1992
3,47.02,NaN,Edwin MOSES,31 AUG 1955,USA,Koblenz (GER),31 AUG 1983
4,47.13,NaN,Edwin MOSES,31 AUG 1955,USA,Milano (ITA),03 JUL 1980


# Scrub Data